I did not do reading notes so this will only be Wednesday Lecture notes

## Index of Coincidence

IC = 26 * [drawing 2 of the same letter if choosing letters at random]

$$
IC = 26 \sum_\alpha \frac{N_\alpha (N_\alpha - 1)}{N(N-1)}
$$
- $N_\alpha \ N$ is draw an alpha
- $(N_\alpha - 1) \ (N - 1)$ is draw another alpha

$$
\approx 26 \sum_\alpha P_\alpha^2
$$

For English plaintext IC is usually:
- ~1.75
- [1.5, 2] ("almost always", not necessarily a concrete confidence interval)

> Drawing from the observation that even smaller samples fall within range, prof says confidence level is most likely within upper 90%

The 26* is just to normalize

Otherwise,,,

For N>>0, moves IC into range [1, 26] instead of [1/26, 1]

IC stays roughly the same if we apply
- Transposition cipher 
    > Just shuffles the letters around. The value is calculated by combining the prob for all the letters. The position of letter does not affect the overall value
- Simple substitution
    > Swaps letter frequencies around. The value is calculated by combining the prob for all letters (regardless of letter). The individual prob of each letter does not affect the overall value.

## Breaking Vignere 

- Guess period (or Kasiski exam)

In [21]:
# Courtesy of https://github.com/nytr0gen/kasiski/blob/master/kasiski.py
import re, math

def normalize(s):
    s = s.strip().upper()
    s = re.sub(r'[^A-Z]+', '', s)
    return s

def kasiski(s, min_num = 3):
    s = normalize(s)
    out = ''

    matches = []
    found = {}
    for k in range(min_num, len(s)):
        found[k] = {}
        shouldbreak = True
        for i in range(0, len(s) - k):
            v = s[i:i+k]
            if v not in found[k]:
                found[k][v] = 1
            else:
                found[k][v] += 1
                shouldbreak = False

        if shouldbreak:
            break

        for v in found[k]:
            if found[k][v] > 2:
                matches.append(v)

    out += "Length  Count  Word        Factor  Location (distance)\n"
    out += "======  =====  ==========  ======  ===================\n"
    keylens = {}
    for v in matches:
        k = len(v)
        p = []
        for i in range(len(s)):
            if s[i:i+k] == v:
                p.append(i)

        # assuming len(p) > 1
        factor = p[1] - p[0]
        for i in range(2, len(p)):
            factor = math.gcd(factor, p[i] - p[i - 1])

        locations = ""
        for i in range(len(p)):
            locations += "%d " % p[i]
            if i > 0:
                locations += "(%d) " % (p[i] - p[i-1])

        out += "%6d  %5d  %10s  %6d  %s\n" % (k, found[k][v], v, factor, locations)

    return found

In [22]:
cipher = """IKGWERTZONXEULTEZWHWXTYHSZKEGQOJNENRUDJGFRNKUISLDZOMKDKWZHVUOSJLFZEJJONQHWVRFEATRYIDIKKDKKEHNAEWOEYFIRMLNIENIFMAJKELXAMHRKKDKKEEUOIVADUNVRNLNEERNKKNJHHWNAUKESXDYLSHGRVQTKGNUFOEVAELOFYRVSESZYVWSLOLCLDGTTCLKWHEZQGGATZQTZKDRUKFUWRQDAJOEWLAQESHIFMLVITTEMPVEDLIEWHAYGILMZUUJHIUGNERTZKLGLTAYHROLTKGCDDONEEWHWELVDVWOTRQDKNIWWTGZHVJRSTGVWHWEAIHGGONXWOLNEXUAFMEKKEFOSRLDQKSRQSOKRVGMJYDVDNSYSFRNSYTYHYSXEDDRJOEUDNVZHRWWARLSHOFTENBESXSUDYSTDNKOOOLCOINKHVUELNEEZHQPOJHPZCICOTSQETDRWUFKKEZUUJHAFJPVUHSVSROAVZOBHEHNIDFOEVAEBTZKYNLLDRIMHIFZHVNILIHVQAFJTYHRWYTNLLDHEJKULAPWRRLNELVEGLSLFHYNOJWSSYCYROKKTFLNZGBZWILOOSVEJBEUQOEXLFFKOUOUVAAJNVOLQYHRNIFMHVUHWGDZEEDOEMHTZKDVDDSXERWPWGCVEULOTZVNGZRZJHLZOJSESQOWWHWSWZWHDKVZWYSZTYDTEUMVQTLNEXDRVKNXDTWYWLQGLUTYHRSSBCHRKCEIHRWZUIQIFMTYHYSXERIRSODFINGZHZQGAMRLPBDKDNDTUNIEJTZKIIDPHXORFHLNRFXGZZHVZIFJONWOYKTYHRLNEPZOMRDSUANKSRWAFGNUDLDNIJOEYOOEVAKZHVBSLKPGHDGTTFWHWJOFUSLUNVVAFJHROTWJTFWACKACDSLROFNALZHVPOGTOIPOJKCFURWITCBALKATKOLNEIEYZKRCLGZZIWHLLORIHSAYTZELQOMGHLDKDKREKIAGHTZKMRJAATAEGPJKSJLNYGRVPEEHRRQCWONKRTZKHRQDGLMIVDWGNRQDVOSIHGSXDZQGZKRVAPGYTLOALOOEVALSYIXDWTEJVINGNZVHWJTYUOMMHKKECOTTKEFGSKKEQUPVQEVZHVKOMYEUROJGNUVOKNOLODZGVVFOFLIIPEVPOJHPZONYLSGVIELOFUFYLSXKLCRWKKRMDNLYGRBIFJIJFRWZIFQSZGDYHNGZFFUTMTAKHLQXETRGFOSVGMWLOIDRWYPVFTSHLVFHSXATWEJHYKKEKCEVWRATGFIAKUVVUEAMNRWHAYFVHTEEWROKZUMVZAKREEJTZKNVGBQGDZYEJYIFQIFZHVGIJKCKLOFUFKKECORBZHWTBVQESZHZWSOGLCVIHKRTHINKDUHCSEHRGMSJEGUOYXEJVENKNZQSWBEEPOFZHJPAFEANLNVUWJKOOKDSOAUQGRSSVKPILVWJOWJLSYSRQDKRAKHSBATKHDGLFYHRWGNUWHWXESHYGTDKKEJOGYWLATEFITZKRFRFLUBVJRSJUROLQCOINEVUFWLNUUMZQGSATLPNKZOIPSAYOLJHLGNUVOGTDZVCGBEIHDLNEKKRWKHVDDKZOEHSGTTYHSDUPVQEPZTYHMGURKKEEODUOEGTEXUEQGNUKADLBLUIWJIEKESZHVGGSXLZQTGTSFQLQNAIPOFOZVGBQZHVWUJLAEGMGYSTUEWVIEJUHOTJIOGZHVDTZILZIFKYTZOLTGRVLLATGVUEVXOLQDLNEDXNVKRKKALHEELGFYKPZALIHVGTZKMFWHKLLLWTWXIEJAEUNXWHWNERWHSTDYDRWHECOSDOSKHNWJTFWHWYOWWWATDSUESZHZQGLNRFXGZZHVJRSYSRQDOUNUHRWJHFZAFEOEHCGALUHVWXIDDGATELQQMOEKVLMSBVUSXURKKEKREVSEJYIEWHSZQLLELKAIWH"""

found = (kasiski(cipher))

{l:{k:v for k,v in t.items() if v > 1} for l,t in found.items() if l>4}

{5: {'KKDKK': 2,
  'KDKKE': 2,
  'FOEVA': 2,
  'OEVAE': 2,
  'ZUUJH': 2,
  'ZHVJR': 2,
  'HVJRS': 2,
  'TYHYS': 2,
  'YHYSX': 2,
  'HYSXE': 2,
  'POJHP': 2,
  'OJHPZ': 2,
  'UFKKE': 2,
  'IFZHV': 2,
  'ZHZQG': 2,
  'EJTZK': 2,
  'LNRFX': 2,
  'NRFXG': 2,
  'RFXGZ': 2,
  'FXGZZ': 2,
  'XGZZH': 2,
  'GZZHV': 2,
  'OOEVA': 2,
  'TFWHW': 2,
  'WJTFW': 2,
  'KKECO': 2,
  'UPVQE': 2,
  'GNUVO': 2,
  'LOFUF': 2,
  'ESZHZ': 2,
  'SYSRQ': 2,
  'YSRQD': 2,
  'URKKE': 2},
 6: {'KKDKKE': 2,
  'FOEVAE': 2,
  'ZHVJRS': 2,
  'TYHYSX': 2,
  'YHYSXE': 2,
  'POJHPZ': 2,
  'LNRFXG': 2,
  'NRFXGZ': 2,
  'RFXGZZ': 2,
  'FXGZZH': 2,
  'XGZZHV': 2,
  'SYSRQD': 2},
 7: {'TYHYSXE': 2, 'LNRFXGZ': 2, 'NRFXGZZ': 2, 'RFXGZZH': 2, 'FXGZZHV': 2},
 8: {'LNRFXGZZ': 2, 'NRFXGZZH': 2, 'RFXGZZHV': 2},
 9: {'LNRFXGZZH': 2, 'NRFXGZZHV': 2},
 10: {'LNRFXGZZHV': 2},
 11: {}}

Ooops, got carried away, we moved on

But just to write this down

with period --> break cipher into columns of length period

Each column is a ceasar cipher

## Q1 

If IC of cipher is 1.1, which of following ciphers was most likely used to encrypt?
- A) Rectangular transposition
- B) Affine
- C) Vignere
- D) Caesar

Well A and D can be crossed out since we discussed they would fall within the [1.5 and 2] range.

Affine is ALSO simple substitution

Vignere I was iffy on
- at first thought it was because you're using another body of text (which would hold similar IC) to encrypt
- but you're NOT, ur using a KEY. Which is does NOT have an IC in that range. 

> Asked prof if IC would approach standard range if key was longer. But prof actually says he believes the IC would get closer to 1 instead... hmm. But he did say he had to think carefully about it

## Q2

Recall alien lang from Quiz 4. 

3 letter alphabet with plaintext freq of (20%, 20%, 60%)

Which would be closest to the expected IC for plaintext in this language?
- A) 1
- B) 1.3
- C) 1.7
- D) 2

$$
\approx 3 * \sum_\alpha p_\alpha^2

$$

In [25]:
(.2**2 + .2**2 + .6**2) * 3

1.32

## Q3 

>oops forgot to write it down

Table of several guesses for period and the calculated IC for each col

Chose the col with most amount of values in standard range